In [12]:
import sys
sys.executable

'C:\\Users\\vietx\\anaconda3\\python.exe'

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Khám phá dữ liệu

### Đọc dữ liệu từ file csv


In [14]:
billboards = pd.read_csv('charts.csv')
billboards.head()

,date,rank,song,artist,last-week,peak-rank,weeks-on-board
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18


### Dữ liệu có bao nhiêu dòng và bao nhiêu cột?

In [15]:
num_rows = len(billboards)
num_cols = len(billboards.columns)

num_rows, num_cols

(330087, 7)

### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?
- Mỗi dòng của dữ liệu là thông tin bài hát và các thông số xếp hạng của bài hát trong bảng xếp hạng trong một tuần
- Dựa theo dữ liệu ta thấy rằng các dòng không có ý nghĩa khác nhau

### Dữ liệu có các dòng bị lặp không?


In [16]:
duplicated = np.sum(billboards.duplicated()) != 0
duplicated

False

### Mỗi cột dữ liệu có ý nghĩa:
- Date: Ngày bảng xếp hạng tuần của Billboard được công bố
- Rank: Thứ hạng của bài hát trên bảng xếp hạng tuần Billboard được công bố vào ngày ở cột đầu tiên.
- Song: Tên bài hát được xếp hạng
- Artist: Ca sỹ hát bài hát
- Last-week: Thứ hạng của bài hát đó trên bảng xếp hạng tuần trước của Billboard.
- Peak-rank: Thứ hạng cao nhất của bài hát đó trên tất cả các bảng xếp hạng tuần của Billboard.
- Weeks-on-board: ...


### Kiểu dữ liệu của mỗi cột

In [18]:
col_dtypes = billboards.dtypes

col_dtypes

date               object
rank                int64
song               object
artist             object
last-week         float64
peak-rank           int64
weeks-on-board      int64
dtype: object


Các cột có kiểu dữ liệu chưa phù hợp:
- Cột date đang ở dạng object, chưa phù hợp để xử lý nên ta cần chuyển về kiểu datetime.
- Cột last-week nên có dữ liệu là int thay vì float64.
- Cột song & artist có kiểu là object nhưng có thể kiểu dữ liệu thật sự là string

# Tiền xử lý

### Chuyển dtype của cột "Date" sang datetime 

In [22]:
billboards['date'] = pd.to_datetime(billboards['date'], format = '%Y-%m-%d')

### Chuyển dtype của cột last-week sang int

In [23]:
billboards['last-week'] = billboards['last-week'].replace(np.nan, 0).astype('int64')

Ta thay thế các ô có giá trị nan bằng 0. Việc thay thế này là hợp lý vì không có thứ hạng 0 trên BXH. Khi đó, ta có thể ngầm hiểu khi cột last-week có giá trị là 0 thì bài hát đó lần đầu lọt vào BXH.

# Khám phá dữ liệu (tiếp tục)

### Với mỗi cột có kiểu dữ liệu dạng số (numerical), các giá trị được phân bố như thế nào?

In [34]:
nume_cols = ['date', 'rank', 'last-week', 'peak-rank', 'weeks-on-board']

def missing_ratio(col):
    return col.isnull().sum() / len(col)

nume_col_profiles_df = billboards.agg(['min', 'max', missing_ratio])[cols]

nume_col_profiles_df

,date,rank,last-week,peak-rank,weeks-on-board
min,1958-08-04 00:00:00,1.0,0.0,1.0,1.0
max,2021-11-06 00:00:00,100.0,100.0,100.0,90.0
missing_ratio,0.0,0.0,0.0,0.0,0.0


Không có gì bất thường với các giá trị min, max, missing_ratio của các cột. Ta có thể kiểm chứng giá trị max của weeks-on-board bằng cách tìm trên google bài hát có mặt trên BXH Billboard 90 tuần. Sau khi tra cứu, ta thấy bài hát đó chính là "Blinding Light" và chính thức rời khỏi BXH vào tháng 9/2021. Khá hợp lý khi BXH của ta có tuần cuối cùng được thống kê vào tháng 11/2021.

### Với mỗi cột có kiểu dữ liệu dạng phân loại (categorical),các giá trị được phân bố như thế nào?

In [35]:
cate_cols = ['artist', 'song']

def num_diff_vals(col):
    return col.nunique()

def diff_vals(col):
    return col[col.isna()==False].unique()

cate_col_profiles_df = billboards.agg([missing_ratio, num_diff_vals, diff_vals])[cate_cols]

cate_col_profiles_df

,artist,song
missing_ratio,0.0,0.0
num_diff_vals,10205,24620
diff_vals,"[Adele, The Kid LAROI & Justin Bieber, Lil Nas...","[Easy On Me, Stay, Industry Baby, Fancy Like, ..."


Ta thấy rằng không có gì quá bất thường. Mising ratio bằng 0 là hợp lý. Mặt khác, một ca sỹ có thể có nhiều bài hát nên việc số lượng bài hát nhiều hơn gấp hơn 2 lần so với số lượng ca sỹ là bình thường.